In [ ]:
import os
import getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

# Memory in LangChain

[Short-term Memory](https://docs.langchain.com/oss/python/langchain/short-term-memory)

<img src="https://www.gettingstarted.ai/content/images/size/w2400/2023/10/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png">


### Reading & Writing: The Two Memory Must-Dos ✍️

1. **Reading**: Memory feeds context into the AI's thought process.
2. **Writing**: Memory saves the latest session details for next time.

*Remember: context-length*
Short term memory lets your application remember previous interactions within a single thread or conversation.

Conversation history is the most common form of short-term memory. Long conversations pose a challenge to today’s LLMs; a full history may not fit inside an LLM’s context window, resulting in an context loss or errors.

Even if your model supports the full context length, most LLMs still perform poorly over long contexts. They get “distracted” by stale or off-topic content, all while suffering from slower response times and higher costs.

*We saw how to add memory in the previous Notebook.*

Chat models accept context using messages, which include instructions (a system message) and inputs (human messages). In chat applications, messages alternate between human inputs and model responses, resulting in a list of messages that grows longer over time. Because context windows are limited, many applications can benefit from using techniques to remove or “forget” stale information.

With short-term memory enabled, long conversations can exceed the LLM’s context window. Common solutions are:

1. [Trim messages](https://docs.langchain.com/oss/python/langchain/short-term-memory#trim-messages)

2. [Delete Messaages](https://docs.langchain.com/oss/python/langchain/short-term-memory#delete-messages)

3. Custom strategies (e.g., message filtering, etc.)

A very common solution is to summarise the previous messages and remember the context summary.


## Example of summarisation 

The problem with trimming or removing messages, as shown above, is that you may lose information from culling of the message queue. Because of this, some applications benefit from a more sophisticated approach of summarizing the message history using a chat model.

![alt text](https://mintcdn.com/langchain-5e9cc07a/ybiAaBfoBvFquMDz/oss/images/summary.png?w=1100&fit=max&auto=format&n=ybiAaBfoBvFquMDz&q=85&s=4abdac693a562788aa0db8681bef8ea7)

In [2]:
# type: ignore
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
from langchain_openai import ChatOpenAI 

model_nano = ChatOpenAI(model="gpt-5-nano-2025-08-07")
model_mini = ChatOpenAI(model="gpt-5-mini-2025-08-07")


checkpointer = InMemorySaver()

agent = create_agent(
    model=model_mini,
    tools=[],
    middleware=[
        SummarizationMiddleware(
            model=model_nano,
            max_tokens_before_summary=4000,  # Trigger summarization at 4000 tokens
            messages_to_keep=20,  # Keep last 20 messages after summary
        )
    ],
    checkpointer=checkpointer,
)

config: RunnableConfig = {"configurable": {"thread_id": "1"}}
agent.invoke({"messages": "hi, my name is Mo"}, config)
agent.invoke({"messages": "write a short poem about cats"}, config)
agent.invoke({"messages": "now do the same but for dogs"}, config)
final_response = agent.invoke({"messages": "what's my name?"}, config)

final_response["messages"][-1].pretty_print()

/Users/mo/IADT_local/25_26/AI/Prompt_Engineering_LangChain/Prompt_Engineering_LangChain/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


================================== Ai Message ==================================

You said your name is Mo.  
Would you like me to call you Mo or something else?
